In [2]:
!pip3 install torch torchvision torchaudio
!pip3 install transformers
from sentence_transformers import SentenceTransformer
from sentence_transformers import util
from tqdm import tqdm
import torch
import json

You should consider upgrading via the '/Users/arush/workspace/646proj/LaMP/data/flant5/bin/python3 -m pip install --upgrade pip' command.
You should consider upgrading via the '/Users/arush/workspace/646proj/LaMP/data/flant5/bin/python3 -m pip install --upgrade pip' command.


/Users/arush/workspace/646proj/LaMP/data/flant5/lib/python3.9/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [3]:
model = SentenceTransformer('msmarco-distilbert-base-dot-prod-v3')

In [4]:
with open("/Users/arush/downloads/IR_LAMP/Personalized_News_Categorization_validation_input.json", "r") as f:
    json_data = json.load(f)

In [5]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
llm_model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-xl")
tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-xl")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [6]:
golds = []
for user in tqdm(json_data):
    corpus = []
    categories = []
    profile = user['profile']
    query = user['input']
    
    input_prompt = "Your task is to answer the query based on user's relevant history interactions. Here’s the query: {}\nHere are user's past interactions: ".format(query)
    past_interactions = []

    for doc in profile:
        corpus.append(doc['text'])
        categories.append(doc['category'])
    corpus_embeddings = model.encode(corpus, convert_to_tensor=True)
    query_embedding = model.encode(query, convert_to_tensor=True)
    hits = util.semantic_search(query_embedding, corpus_embeddings, top_k=3, score_function=util.dot_score)
    hits = hits[0]
    predictions = {}
    
    for hit in hits:
        past_interactions.append("User categorized the news: '{}' into '{}'".format(corpus[hit['corpus_id']], categories[hit['corpus_id']]))
    input_prompt += "\n".join(past_interactions)
    inputs = tokenizer(input_prompt, return_tensors="pt")
    outputs = llm_model.generate(**inputs)
    predictions["output"] = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]
    predictions["id"] = user["id"]
    golds.append(predictions)

final_json = {}
final_json["task"] = "LaMP_2"
final_json["golds"] = golds

  0%|                                                  | 0/1052 [00:00<?, ?it/s]/Users/arush/workspace/646proj/LaMP/data/flant5/lib/python3.9/site-packages/transformers/generation/utils.py:1273: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
  0%|                                                  | 0/1052 [01:56<?, ?it/s]


In [7]:
print(final_json)

{'task': 'LaMP_2', 'golds': [{'output': 'politics', 'id': '110'}]}


In [15]:
with open("pred.json", "w") as f:
    json.dump(final_json, f)